In [1]:
import argparse
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.metrics import accuracy_score
import tqdm

from torch.utils.data import Subset
from torchvision.datasets import MNIST, SVHN, USPS

In [2]:
from src.lib import *

공격 준비를 위한 transform, dataset, loader 설정

In [3]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# define transforms
transform = transforms.Compose([
        transforms.ToTensor(),
        normalize
])

dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_set, valid_set= torch.utils.data.random_split(dataset, [40000,10000])
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=64, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


NOH

Dataset; CIFAR10

Victim; MobileNet

Shadow; SqueezeNet(pretrained)

In [4]:
clientMobilenet = ClientMobileNet()
serverMobilenet = ServerMobileNet()
clientMobilenet.load_state_dict(torch.load('./assets/mobilenet_cifar10_c0.pth'))
serverMobilenet.load_state_dict(torch.load('./assets/mobilenet_cifar10_server.pth'))

<All keys matched successfully>

In [10]:
num_epoch = 50
shadow_client = ClientMobileNet()
shadow_server = ServerMobileNet()
shadow_client.load_state_dict(torch.load('./assets/mobilenet_cifar10_c1.pth'))
optimizer = optim.Adam(shadow_server.parameters())
criterion = nn.CrossEntropyLoss()
device = 'cpu'
shadow_server.train()
softmax = nn.Softmax(dim=1)
for epoch in range(num_epoch):
    total_loss = 0
    for x,y in test_loader:
        y_label = softmax(serverMobilenet(clientMobilenet(x)).detach())
        y_shadow = shadow_server(shadow_client(x))
        optimizer.zero_grad()
        loss = criterion(y_shadow, y_label)
        total_loss += loss
        loss.backward()
        optimizer.step()
    print(f"epoch {(epoch+1)}: loss {total_loss}")
print("done")
    

epoch 1: loss 360.8634948730469
epoch 2: loss 355.38128662109375
epoch 3: loss 356.6088562011719
epoch 4: loss 354.1479187011719
epoch 5: loss 353.891357421875
epoch 6: loss 353.01800537109375
epoch 7: loss 351.36688232421875
epoch 8: loss 350.6422424316406
epoch 9: loss 350.6971435546875
epoch 10: loss 350.6357116699219
epoch 11: loss 350.5658874511719
epoch 12: loss 350.47454833984375
epoch 13: loss 349.5735168457031
epoch 14: loss 349.07159423828125
epoch 15: loss 348.295654296875
epoch 16: loss 348.69012451171875
epoch 17: loss 348.0174865722656
epoch 18: loss 347.276611328125
epoch 19: loss 347.5107727050781
epoch 20: loss 346.2996520996094
epoch 21: loss 345.5970458984375
epoch 22: loss 345.6961669921875
epoch 23: loss 345.3965148925781
epoch 24: loss 344.4766540527344
epoch 25: loss 344.8792419433594
epoch 26: loss 344.2704772949219
epoch 27: loss 343.13995361328125
epoch 28: loss 343.29827880859375
epoch 29: loss 342.33056640625
epoch 30: loss 341.3404541015625
epoch 31: loss 3

In [6]:
shadow_server.eval()
acc, tot = 0, 0
for x, y in train_loader:
    y_pred = shadow_server(shadow_client(x))
    acc += (y==y_pred.argmax(1)).sum()
    tot += len(y)
print(f"test acc : {acc}/{tot}")

test acc : 5999/10000


In [12]:
torch.save(shadow_server,'mobile_cifar10_partial_squeezenet.pth')

NOH

Dataset; CIFAR10

Victim; MobileNet

Shadow; SqueezeNet(pretrained)

In [6]:
clientMobilenet = ClientMobileNet()
serverMobilenet = ServerMobileNet()
clientMobilenet.load_state_dict(torch.load('./assets/mobilenet_cifar10_c0.pth'))
serverMobilenet.load_state_dict(torch.load('./assets/mobilenet_cifar10_server.pth'))

num_epoch = 50
shadow_client = ClientSqueezeNet()
shadow_server = ServerSqueezeNet()
shadow_client.load_state_dict(torch.load('./assets/squeezenet_cifar10_c0.pth'))
optimizer = optim.Adam(shadow_server.parameters())
criterion = nn.CrossEntropyLoss()
device = 'cpu'
shadow_server.train()
softmax = nn.Softmax(dim=1)
for epoch in range(num_epoch):
    total_loss = 0
    for x,y in test_loader:
        y_label = softmax(serverMobilenet(clientMobilenet(x)).detach())
        y_shadow = softmax(shadow_server(shadow_client(x)))
        optimizer.zero_grad()
        loss = criterion(y_shadow, y_label)
        total_loss += loss
        loss.backward()
        optimizer.step()
    print(f"epoch {(epoch+1)}: loss {total_loss}")
print("done")

shadow_server.eval()
acc, tot = 0, 0
for x, y in valid_loader:
    y_pred = shadow_server(shadow_client(x))
    acc += (y==y_pred.argmax(1)).sum()
    tot += len(y)
print(f"test acc : {acc}/{tot}")

torch.save(shadow_server,'mobile_cifar10_partial_mobilenet.pth')

epoch 1: loss 347.5904541015625
epoch 2: loss 332.9650573730469
epoch 3: loss 327.3974914550781
epoch 4: loss 323.4686584472656
epoch 5: loss 321.5416564941406
epoch 6: loss 318.6306457519531
epoch 7: loss 316.3725891113281
epoch 8: loss 313.9549560546875
epoch 9: loss 310.8152770996094
epoch 10: loss 309.2518310546875
epoch 11: loss 308.26129150390625
epoch 12: loss 306.70794677734375
epoch 13: loss 305.8299865722656
epoch 14: loss 305.0028381347656
epoch 15: loss 302.9578552246094
epoch 16: loss 301.7199401855469
epoch 17: loss 302.11187744140625
epoch 18: loss 300.0210266113281
epoch 19: loss 298.55059814453125
epoch 20: loss 298.4129333496094
epoch 21: loss 297.7160949707031
epoch 22: loss 299.9552307128906
epoch 23: loss 296.4400634765625
epoch 24: loss 295.4084167480469
epoch 25: loss 295.0885009765625
epoch 26: loss 295.8665771484375
epoch 27: loss 294.68072509765625
epoch 28: loss 294.2745666503906
epoch 29: loss 294.9202575683594
epoch 30: loss 294.990966796875
epoch 31: loss 

NOH

Victim: SqueezeNet => all failed

In [ ]:
clientMobilenet = ClientSqueezeNet()
serverMobilenet = ServerSqueezeNet()
clientMobilenet.load_state_dict(torch.load('./assets/squeezenet_cifar10_c0.pth'))
serverMobilenet.load_state_dict(torch.load('./assets/squeezenet_cifar10_server.pth'))

num_epoch = 20
shadow_client = ClientMobileNet()
shadow_server = ServerMobileNet()
shadow_client.load_state_dict(torch.load('./assets/mobilenet_cifar10_c0.pth'))
optimizer = optim.Adam(shadow_server.parameters())
criterion = nn.CrossEntropyLoss()
device = 'cpu'
shadow_server.train()
softmax = nn.Softmax(dim=1)
for epoch in range(num_epoch):
    total_loss = 0
    for x,y in train_loader:
        y_label = softmax(serverMobilenet(clientMobilenet(x)).detach())
        y_shadow = shadow_server(shadow_client(x))
        optimizer.zero_grad()
        loss = criterion(y_shadow, y_label)
        total_loss += loss
        loss.backward()
        optimizer.step()
    print(f"epoch {(epoch+1)}: loss {total_loss}")
print("done")

shadow_server.eval()
acc, tot = 0, 0
for x, y in valid_loader:
    y_pred = shadow_server(shadow_client(x))
    acc += (y==y_pred.argmax(1)).sum()
    tot += len(y)
print(f"test acc : {acc}/{tot}")

torch.save(shadow_server,'squeeze_cifar10_partial_mobilenet.pth')

In [4]:
clientMobilenet = ClientSqueezeNet()
serverMobilenet = ServerSqueezeNet()
clientMobilenet.load_state_dict(torch.load('./assets/squeezenet_cifar10_c0.pth'))
serverMobilenet.load_state_dict(torch.load('./assets/squeezenet_cifar10_server.pth'))

num_epoch = 20
shadow_client = ClientSqueezeNet()
shadow_server = ServerSqueezeNet()
shadow_client.load_state_dict(torch.load('./assets/squeezenet_cifar10_c0.pth'))
optimizer = optim.Adam(shadow_server.parameters())
criterion = nn.CrossEntropyLoss()
device = 'cpu'
shadow_server.train()
softmax = nn.Softmax(dim=1)
for epoch in range(num_epoch):
    total_loss = 0
    for x,y in train_loader:
        y_label = softmax(serverMobilenet(clientMobilenet(x)).detach())
        y_shadow = shadow_server(shadow_client(x))
        optimizer.zero_grad()
        loss = criterion(y_shadow, y_label)
        total_loss += loss
        loss.backward()
        optimizer.step()
    print(f"epoch {(epoch+1)}: loss {total_loss/64}")
print("done")

shadow_server.eval()
acc, tot = 0, 0
for x, y in valid_loader:
    y_pred = shadow_server(shadow_client(x))
    acc += (y==y_pred.argmax(1)).sum()
    tot += len(y)
print(f"test acc : {acc}/{tot}")

torch.save(shadow_server,'squeeze_cifar10_partial_mobilenet.pth')

epoch 1: loss 20.66016960144043
epoch 2: loss 20.529081344604492
epoch 3: loss 20.50014877319336
epoch 4: loss 20.478044509887695
epoch 5: loss 20.482629776000977
epoch 6: loss 20.452669143676758
epoch 7: loss 20.445524215698242
epoch 8: loss 20.44179344177246
epoch 9: loss 20.430282592773438
epoch 10: loss 20.422414779663086
epoch 11: loss 20.42481231689453
epoch 12: loss 20.41050148010254
epoch 13: loss 20.405685424804688
epoch 14: loss 20.407556533813477
epoch 15: loss 20.401308059692383
epoch 16: loss 20.403257369995117
epoch 17: loss 20.39028549194336
epoch 18: loss 20.396442413330078
epoch 19: loss 20.390295028686523
epoch 20: loss 20.39445686340332
done
test acc : 1048/10000


In [4]:
clientSqueezenet = ClientSqueezeNet()
serverSqueezenet = ServerSqueezeNet()
clientSqueezenet.load_state_dict(torch.load('./assets/squeezenet_cifar10_c0.pth'))
serverSqueezenet.load_state_dict(torch.load('./assets/squeezenet_cifar10_server.pth'))

num_epoch = 20
shadow = MobileNet()
optimizer = optim.Adam(shadow.parameters())
criterion = nn.CrossEntropyLoss()
device = 'cpu'
shadow.train()
softmax = nn.Softmax(dim=1)
for epoch in range(num_epoch):
    total_loss = 0
    for x,y in train_loader:
        y_label = serverSqueezenet(clientSqueezenet(x)).detach()
        y_shadow = shadow(x)
        optimizer.zero_grad()
        loss = criterion(y_shadow, y_label)
        total_loss += loss
        loss.backward()
        optimizer.step()
    print(f"epoch {(epoch+1)}: loss {total_loss}")
print("done")

shadow.eval()
acc, tot = 0, 0
for x, y in valid_loader:
    y_pred = shadow(x)
    acc += (y==y_pred.argmax(1)).sum()
    tot += len(y)
print(f"test acc : {acc}/{tot}")

torch.save(shadow,'squeeze_cifar10_whole_mobilenet.pth')

epoch 1: loss 4940.9765625
epoch 2: loss 4848.1787109375
epoch 3: loss 4819.31982421875
epoch 4: loss 4796.330078125
epoch 5: loss 4797.45751953125
epoch 6: loss 4777.63427734375
epoch 7: loss 4785.51318359375
epoch 8: loss 4773.89501953125
epoch 9: loss 4770.45361328125
epoch 10: loss 4758.4453125
epoch 11: loss 4765.1181640625
epoch 12: loss 4752.99658203125
epoch 13: loss 4763.14501953125
epoch 14: loss 4755.58056640625
epoch 15: loss 4748.89111328125
epoch 16: loss 4743.9208984375
epoch 17: loss 4743.8349609375


In [4]:
clientSqueezenet = ClientSqueezeNet()
serverSqueezenet = ServerSqueezeNet()
clientSqueezenet.load_state_dict(torch.load('./assets/squeezenet_cifar10_c0.pth'))
serverSqueezenet.load_state_dict(torch.load('./assets/squeezenet_cifar10_server.pth'))

num_epoch = 50
shadow_client = ClientSqueezeNet()
shadow_server = ServerSqueezeNet()
shadow_client.load_state_dict(torch.load('./assets/squeezenet_cifar10_c0.pth'))
optimizer = optim.Adam(shadow_server.parameters())
criterion = nn.CrossEntropyLoss()
device = 'cpu'
shadow_server.train()
softmax = nn.Softmax(dim=1)
for epoch in range(num_epoch):
    total_loss = 0
    for x,y in test_loader:
        y_label = softmax(serverSqueezenet(clientSqueezenet(x)).detach())
        y_shadow = softmax(shadow_server(shadow_client(x)))
        optimizer.zero_grad()
        loss = criterion(y_shadow, y_label)
        total_loss += loss
        loss.backward()
        optimizer.step()
    print(f"epoch {(epoch+1)}: loss {total_loss}")
print("done")

shadow_server.eval()
acc, tot = 0, 0
for x, y in valid_loader:
    y_pred = shadow_server(shadow_client(x))
    acc += (y==y_pred.argmax(1)).sum()
    tot += len(y)
print(f"test acc : {acc}/{tot}")

torch.save(shadow_server,'squeeze_cifar10_whole_mobilenet.pth')

epoch 1: loss 347.0039978027344
epoch 2: loss 346.6831359863281
epoch 3: loss 346.561279296875
epoch 4: loss 346.5929260253906
epoch 5: loss 346.7683410644531
epoch 6: loss 346.840576171875
epoch 7: loss 346.7769470214844
epoch 8: loss 346.7422180175781
epoch 9: loss 346.4700622558594
epoch 10: loss 346.7978515625
epoch 11: loss 346.7279357910156
epoch 12: loss 346.5966796875
epoch 13: loss 346.72064208984375
epoch 14: loss 346.5398864746094
epoch 15: loss 346.50396728515625
epoch 16: loss 346.6377258300781
epoch 17: loss 346.6171569824219
epoch 18: loss 346.6437072753906
epoch 19: loss 346.6429138183594
epoch 20: loss 346.8576354980469
epoch 21: loss 346.5480651855469
epoch 22: loss 346.8531188964844
epoch 23: loss 346.63702392578125
epoch 24: loss 346.6578063964844
epoch 25: loss 346.5018005371094
epoch 26: loss 346.5928039550781
epoch 27: loss 346.57183837890625
epoch 28: loss 346.64581298828125
epoch 29: loss 346.8637390136719
epoch 30: loss 346.7839660644531
epoch 31: loss 346.762

NameError: name 'shadow' is not defined